# Introduction

This workout comes from [2023 Week 16 | Power BI: Measure Format Strings and Conditionally Formatted Titles](https://workout-wednesday.com/pbi-2023-w16/), the main focus of which is a dashboard allowing the user to choose the number of top products to be included in the report and the metric to be used in the report visuals. 

In [1]:
import pandas as pd

In [2]:
file_path = '../data/Dataset-Customer Profitability.xlsx'
data = pd.read_excel(file_path, sheet_name=None)
data.keys()

dict_keys(['state', 'executive', 'industry', 'customer', 'product', 'scenario', 'date', 'bu', 'factsales'])

In [3]:
top_num = 5
metrics = '# of Customers'

# def get_data(top_num, metrics):
customers = data['customer'].merge(
    data['industry'], left_on='Industry ID', right_on='ID', how='left').merge(
    data['state'], left_on='State', right_on='StateCode', how='left')
data_customer = data['factsales'].merge(
    data['product'], on='Product Key', how='left').merge(
    customers, left_on='Customer Key', right_on='Customer', how='left').merge(
    data['date'], on='YearPeriod', how='left')
data_customer['COGS'] = data_customer.iloc[:,range(6, 12)].sum(axis=1)
data_customer['Gross Margin'] = data_customer['Revenue'] - data_customer['COGS']
data_selected = data_customer.loc[:, ['YearPeriod', 'Customer Key', 'Industry', 'Product', 'State_y', 'COGS', 'Gross Margin', 'Year', 'Qtr', 'Month']]
data_selected['Product'] = data_selected['Product'].fillna('<?>')
data_selected['Industry'] = data_selected['Industry'].fillna('<?>')
# if metrics == '# of Customers':
table_data = data_selected.groupby('Product', dropna=False).nunique().sort_values(by='Customer Key', ascending=False)[['Customer Key']]
indx = table_data.index[:5]
filtered_rows = data_selected['Product'].apply(lambda x: x in indx)
bar_data = data_selected.loc[filtered_rows, :].groupby('Industry', dropna=False).nunique().sort_values(by='Customer Key', ascending=False)[['Customer Key']]
line_data = data_selected.loc[filtered_rows, :].groupby(['Year', 'Qtr', 'Month'], dropna=False).nunique()[['Customer Key']]
state_data = data_selected.loc[filtered_rows, :].groupby('State_y').nunique()[['Customer Key']]

In [18]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
# from bokeh.resources import INLINE
from bokeh.models import ColumnDataSource, DataTable, TableColumn, CDSView, IndexFilter, NumberFormatter, StringFormatter, InlineStyleSheet

output_notebook()

Loading BokehJS ...

In [34]:
source = ColumnDataSource(data_selected)
source_table = ColumnDataSource(table_data)
source_bar = ColumnDataSource(bar_data)
source_line = ColumnDataSource(line_data)
source_map = ColumnDataSource(state_data)

# Table
columns = [
        TableColumn(field="Product", title="Product",),
        TableColumn(field="Customer Key", title="Value",
                    formatter=NumberFormatter(text_align='right')
                   ),
    ]
data_table = DataTable(source=source_table, 
                       view=CDSView(filter=IndexFilter(list(range(5)))),
                       columns=columns, width=300, height=150,
                       index_position=None,
                      )

table_style = InlineStyleSheet(css="""
    .slick-header-columns {
        font-family: arial;
        font-weight: bold;
        font-size: 12pt;
        text-align: right;
    }
    .slick-row {
        font-size: 16pt;
        font-family: arial;
        text-align: right;
    }
""")

data_table.stylesheets = [table_style]

show(data_table)